# Volvo (nome do grupo)

Estimativa de Valores de Horímetro das Máquinas da frota circulante

**Drive**: https://drive.google.com/drive/folders/1HS80lS_XzGH7Tz9juk08d2IR65zZ4Cyv

**Trello**: 

**Github**:

## 0 - Informações sobre o projeto (contexto)

## 1 - Import libs

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

## 2 - Read data

In [ ]:
# Read data
df_Eq = pd.read_excel('../Data/23w06_Equipamentos.xlsx')
df_ST = pd.read_excel('../Data/23w06_SeriesTemporais_Horimetros.xlsx', header = 3)

# Rename columns
df_Eq.rename(columns = {'Chassis ID' : 'ID'}, inplace = True)
df_ST.rename(columns = {'Chassis Id*' : 'ID',
                        'Machine History Date Day' : 'Day',
                        'Compensated Working Hours By Day': 'Time' }, inplace = True)

# Drop columns
df_ST.drop('Unnamed: 0', axis = 1,inplace = True)

## 3 - Business Understanding / Data understanding

### Hipóteses

* H0: É possível prever o horímetro da máquina com base no historico de resgistros com
precisão significativamente maior que a fornecida pelo cálculo atual (baseado em médias)

* H1: É possivel utilizar a mesma técnica para os 3 cenários de medição do problema:
Máquinas que enviaram dados nos últimos 4 dias; máquinas que enviaram dados mas não
recententemente; máquinas que nunca enviaram dados.

* H2: A técnica proposta garante resultados independentemente da classe da máquina:
AHL; WL; EXC

* H3: A técnica proposta garante resultados independentemente do segmento de aplicação
da máquina.

* H4: A técnica proposta garante resultados independentemente do país onde a máquina
está localizada

* H5: A técnica proposta garante resultados independentemente do Dealer que realizou a
venda

* H6: A técnica proposta garante resultados independentemente da idade da máquina

### Objetivo:

**Impacto do Projeto**

***Melhor Gestão da Frota Circulante***
* Aumento da eficiência operacional da rede de distribuição
* Antecipação de necessidades dos Clientes
* Aumento da Produtividade das Máquinas (Uptime)
* Identificação de novas oportunidades em vendas de Peças e Serviços não são exploradas
* Aumento de satisfação dos clientes Volvo CE

In [ ]:
df_Eq.head()

,Região,País,Dealer,Segmento,Modelo,ID,PIN17,Delivery Date,Obs.:,Estado,Região.1
0,Hispânico Sul,Argentina,EPSA,Mining,ART,A040F320538,VCEA040FCG0320538,2020-11-26,NaN,San Luis,Argentina
1,Hispânico Sul,Argentina,EPSA,Construction,ART,A30G732052,VCE0A30GCL0732052,2020-06-22,NaN,San Juan,Argentina
2,Hispânico Sul,Argentina,EPSA,Mining,ART,A30G732029,NaN,2019-11-27,Não encontramos com os métodos de pesquisa uti...,Entre Ríos,Argentina
3,Hispânico Sul,Argentina,EPSA,Mining,EXC,EC210282764,VCEC210DH00282764,2019-06-06,NaN,0,Argentina
4,Hispânico Sul,Argentina,EPSA,Forestry,EXC,EC210282811,VCEC210DV00282811,2019-09-13,Não encontramos com os métodos de pesquisa uti...,0,Argentina


In [ ]:
df_ST.head()

,ID,Day,Time
0,A030F720057,2016-12-09,0.00
1,A030F720057,2016-12-10,5.75
2,A030F720057,2016-12-11,8.95
3,A030F720057,2016-12-12,1.60
4,A030F720057,2016-12-13,8.20


In [ ]:
df_Eq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Região         63 non-null     object        
 1   País           63 non-null     object        
 2   Dealer         63 non-null     object        
 3   Segmento       63 non-null     object        
 4   Modelo         63 non-null     object        
 5   ID             63 non-null     object        
 6   PIN17          57 non-null     object        
 7   Delivery Date  62 non-null     datetime64[ns]
 8   Obs.:          15 non-null     object        
 9   Estado         63 non-null     object        
 10  Região.1       63 non-null     object        
dtypes: datetime64[ns](1), object(10)
memory usage: 5.5+ KB


In [ ]:
df_ST.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118161 entries, 0 to 118160
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   ID      118161 non-null  object        
 1   Day     118161 non-null  datetime64[ns]
 2   Time    118161 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 2.7+ MB


In [ ]:
df_Eq.describe()

C:\Users\aurel\AppData\Local\Temp\ipykernel_2176\4270243714.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_Eq.describe()


,Região,País,Dealer,Segmento,Modelo,ID,PIN17,Delivery Date,Obs.:,Estado,Região.1
count,63,63,63,63,63,63,57,62,15,63,63
unique,3,3,7,5,3,63,57,60,12,29,7
top,Brasil,Brasil,Tecnoeste,Mining,ART,A040F320538,VCEA040FCG0320538,2018-06-15 00:00:00,Não existe articulado na Dinâmica no CareTrack.,Minas Gerais,Sudeste
freq,45,45,10,23,21,1,1,2,3,9,14
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-03-09 00:00:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-19 00:00:00,NaN,NaN,NaN


In [ ]:
df_ST.describe()

,Time
count,118161.000000
mean,5.803329
std,6.365152
min,0.000000
25%,0.038235
50%,4.200000
75%,9.200000
max,85.450000


In [ ]:
df_Eq.ID.nunique()

63

In [ ]:
df_ST.ID.nunique()

62

In [ ]:
id_Eq = df_Eq.ID.unique().tolist()
id_ST = df_ST.ID.unique().tolist()

In [ ]:
df_Eq.query('ID != @id_ST')

,Região,País,Dealer,Segmento,Modelo,ID,PIN17,Delivery Date,Obs.:,Estado,Região.1
47,Brasil,Brasil,Tracbel,Mining,ART,A30F720054,VCEA030FLF0720054,2016-10-13,NaN,Minas Gerais,Sudeste


In [ ]:
df_ST.query('ID != @id_Eq')

,ID,Day,Time


In [ ]:
@interact(agg_fun = ['sum', 'mean','max'])

def funcao(agg_fun):
    df = df_ST.groupby('ID', sort = 'Time')[['Time']].agg(agg_fun)\
              .reset_index()\
              .sort_values('Time', ascending = False)\
              .head(25)
    
    return df

interactive(children=(Dropdown(description='agg_fun', options=('sum', 'mean', 'max'), value='sum'), Output()),…

In [ ]:
@interact(id = df_ST.ID.unique().tolist())
def plot(id):
    plt.figure(figsize= (16,3))
    ax = sns.lineplot(data = df_ST[df_ST.ID == id],
                      x = 'Day',
                      y = 'Time')   
    plt.show()


interactive(children=(Dropdown(description='id', options=('A030F720057', 'A040F320538', 'A040F320657', 'A040F3…

# Observações / ideias 

* Quantidade de ID's diferente nos conjuntos de dados
* Casos com quantidade de hora trabalhada superior a 24hrs